In [16]:
import tensorflow as tf
import os
import matplotlib.pyplot as plt
import numpy as np
import cv2
import pathlib
import tensorflow_hub as hub
import PIL.Image as Image

In [17]:
Image_Shape=(224, 244)

In [18]:
URL_dataset="https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz"

In [19]:
data_dir=tf.keras.utils.get_file(origin=URL_dataset, fname='flower_photos', untar=True)

In [27]:
data_dir=pathlib.Path(data_dir)
image_count=len(list(data_dir.glob('*/*.jpg')))
print(image_count)

3670


In [28]:
# data_dir=pathlib.Path(data_dir)
# file_list=list(data_dir.glob('*/*.jpg'))
# half_file_list=file_list[:len(file_list)//2]
# image_count=len(half_file_list)
# print(image_count)

In [56]:
flowers_images_dict={
    "daisy": list(data_dir.glob('daisy/*')),
    "dandelion": list(data_dir.glob('dandelion/*')),
    "roses": list(data_dir.glob('roses/*')),
    "sunflowers": list(data_dir.glob('sunflowers/*')),
    "tulips": list(data_dir.glob('tulips/*'))
}

flowers_labels_dict={
    "daisy":0,
    "dandelion":0,
    "roses":0,
    "sunflowers":0,
    "tulips":0,
}

X, Y = [],[]

for flower_name, images in flowers_images_dict.items():
    for image in images:
        img=cv2.imread(str(image))
        resized_img=cv2.resize(img, Image_Shape)
        X.append(resized_img)
        Y.append(flowers_labels_dict[flower_name])
X=np.array(X)
y=np.array(Y)

In [57]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test=train_test_split(X, y, random_state=0)
X_train_scaled=X_train/255
X_test_scaled=X_test/255

In [58]:
tf_model = "https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4"

In [ ]:
classifier=tf.keras.Sequential([
    hub.KerasLayer(tf_model, input_shape=(224, 224, 3), trainable=False),
    tf.keras.layers.Dense(len(flowers_labels_dict), activation="softmax")
])
classifier.summary()
classifier.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
metrics=["accuracy"]
)
classifier.fit(X_train_scaled, y_train, epochs=5)
classifier.evaluate(X_test_scaled, y_test)

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer_4 (KerasLayer)  (None, 1280)              2257984   
                                                                 
 dense_4 (Dense)             (None, 5)                 6405      
                                                                 
Total params: 2264389 (8.64 MB)
Trainable params: 6405 (25.02 KB)
Non-trainable params: 2257984 (8.61 MB)
_________________________________________________________________


In [ ]:
from PIL import Image
img = Image.open("roses3.jpg")
img

In [ ]:
img = tf.keras.preprocessing.image.img_to_array(img.resize(Image_Shape))
img = np.array([img])

In [ ]:
res = classifier.predict(img)

In [ ]:
print("The prediction is : {}".format(list(flowers_labels_dict.keys())[np.argmax(res)]))
